In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

In [5]:
df = pd.read_csv('data/creditcard.csv')

In [6]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
df.duplicated().sum()

1081

In [15]:
df=df.drop_duplicates()

In [16]:
df['Class'].value_counts()

Class
0    283253
1       473
Name: count, dtype: int64

Since there is a significant data imbalance which can cause effect the predictions we need to handle the imbalance.

In this scenario we can handle it by creating resampling pipeline where we first oversample category 1 to the average_count and then undersample category 0 to the average_count

Methods like Random Sampling and SMOTE can be used.
- Random sampling involves randomly duplicating samples from the minority class or randomly deleting samples from the majority class. It is simple but it may lead to overfitting during oversampling as it adds duplicate data

- SMOTE creates synthetic samples by interpolating between existing minority class samples. It addresses the overfitting issue associated with random oversampling and can help avoid model bias.

In [17]:
# Splitting the data into features and target
X = df.drop(['Class'], axis=1)   
y = df['Class']

# Get the class distribution
count_0 = y.value_counts()[0]
count_1 = y.value_counts()[1]
average_count = int((count_0+ count_1 ) / 2)

# Create the resampling pipeline
resampling_pipeline = Pipeline([
    ('oversampler', SMOTE(sampling_strategy={1: average_count})),
    ('undersampler', RandomUnderSampler(sampling_strategy={0: average_count}))
])


X_resampled, y_resampled = resampling_pipeline.fit_resample(X, y)

X,y =X_resampled, y_resampled

# Combine the resampled features and target into a DataFrame
df_resampled = pd.concat([X,y],axis=1)
df_resampled.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
79314,58095.0,-1.004327,0.283425,3.086166,1.735252,-0.769330,0.603624,-0.429257,0.516776,0.735310,...,0.149624,0.884531,-0.224911,0.661515,0.400988,0.073012,0.398120,0.159166,30.21,0
261171,160332.0,1.516007,-0.563793,-0.599671,4.010878,-0.210581,0.764687,-0.252287,0.323780,-0.012936,...,0.180650,0.198750,-0.117889,-0.562981,-0.039284,0.135328,-0.063429,-0.036240,188.47,0
253721,156846.0,0.189571,0.863567,0.027969,-0.432262,0.766508,-1.098999,1.273843,-0.507451,0.103837,...,0.322977,1.394574,-0.173282,-0.038050,-0.815125,-0.241871,0.350570,0.098680,3.87,0
120090,75868.0,-1.492824,0.964797,1.827157,-0.067511,0.589997,-0.108777,1.052418,-0.496254,0.104745,...,-0.187434,-0.189701,0.063041,0.156323,-0.658246,-1.063578,-1.424517,-0.754307,8.50,0
197805,132523.0,2.004220,-0.238680,-1.047738,0.330213,-0.258602,-0.896256,-0.056099,-0.063167,0.552158,...,-0.247524,-0.697332,0.367366,-0.032284,-0.416649,0.184569,-0.083155,-0.073844,1.29,0


In [18]:
df_resampled['Class'].value_counts()

Class
0    141863
1    141863
Name: count, dtype: int64

Feature Scaling

In [22]:
scaler = MinMaxScaler()
df_resampled['Time'] = scaler.fit_transform(df_resampled[['Time']])
df_resampled.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
79314,0.336213,-1.004327,0.283425,3.086166,1.735252,-0.769330,0.603624,-0.429257,0.516776,0.735310,...,0.149624,0.884531,-0.224911,0.661515,0.400988,0.073012,0.398120,0.159166,30.21,0
261171,0.927890,1.516007,-0.563793,-0.599671,4.010878,-0.210581,0.764687,-0.252287,0.323780,-0.012936,...,0.180650,0.198750,-0.117889,-0.562981,-0.039284,0.135328,-0.063429,-0.036240,188.47,0
253721,0.907716,0.189571,0.863567,0.027969,-0.432262,0.766508,-1.098999,1.273843,-0.507451,0.103837,...,0.322977,1.394574,-0.173282,-0.038050,-0.815125,-0.241871,0.350570,0.098680,3.87,0
120090,0.439071,-1.492824,0.964797,1.827157,-0.067511,0.589997,-0.108777,1.052418,-0.496254,0.104745,...,-0.187434,-0.189701,0.063041,0.156323,-0.658246,-1.063578,-1.424517,-0.754307,8.50,0
197805,0.766951,2.004220,-0.238680,-1.047738,0.330213,-0.258602,-0.896256,-0.056099,-0.063167,0.552158,...,-0.247524,-0.697332,0.367366,-0.032284,-0.416649,0.184569,-0.083155,-0.073844,1.29,0
